In [2]:
import numpy as np
import pandas as pd
from sklearn import cross_validation
from sklearn import neighbors
from sklearn import grid_search
from sklearn import metrics
from sklearn import linear_model

import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="whitegrid", font_scale=1)
%matplotlib inline

In [3]:
#acquire the data

# Load data
listings = pd.read_csv('listings.csv',sep=',',header=0)#df is my dataframe 

# Check head
listings.head()

/Users/DiegoR/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (43,88) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,9147025,https://www.airbnb.com/rooms/9147025,20160601225828,2016-06-02,Cozy City Island Cottage,City Island is a small community at the edge o...,This cozy little cottage is an open floor plan...,City Island is a small community at the edge o...,none,City Island is famous for it's seafood restaur...,...,9,f,NaN,NaN,f,moderate,f,f,1,1.69
1,6627449,https://www.airbnb.com/rooms/6627449,20160601225828,2016-06-02,Large 1 BDRM in Great location,This ground floor apartment is light and airy ...,"We are close to fishing, boating, biking, hors...",This ground floor apartment is light and airy ...,none,City Island is a unique and a hidden gem of Ne...,...,9,f,NaN,NaN,f,flexible,f,f,1,0.81
2,11675715,https://www.airbnb.com/rooms/11675715,20160601225828,2016-06-02,Cozy 1 BR Basement Apartment,"City Island is an active, year-round nautical ...",This very cozy apartment with features a queen...,"City Island is an active, year-round nautical ...",none,City Island is a small island and a neighborho...,...,10,f,NaN,NaN,t,flexible,f,f,1,4.46
3,8053533,https://www.airbnb.com/rooms/8053533,20160601225828,2016-06-02,Spacious Duplex Apartment,"Duplex apartment with fireplace, two balconies...",NaN,"Duplex apartment with fireplace, two balconies...",none,City Island is one of the gems of NYC. We are...,...,10,f,NaN,NaN,t,flexible,f,f,1,0.63
4,1886820,https://www.airbnb.com/rooms/1886820,20160601225828,2016-06-02,Quaint City Island Community.,Quiet island boating town on Long Island Soun...,"Master bed with queen bed, full bath and offi...",Quiet island boating town on Long Island Soun...,none,Small New England type town in the middle of ...,...,NaN,f,NaN,NaN,f,strict,f,f,1,NaN


In [39]:
#Parse, Mine, Refine data
# Check columns
#list out what the columns are and what their datatype is
for row in listings:
    print(row[0:40])#this will print all the series titles in row 0
    

id
listing_url
scrape_id
last_scraped
name
summary
space
description
experiences_offered
neighborhood_overview
notes
transit
access
interaction
house_rules
thumbnail_url
medium_url
picture_url
xl_picture_url
host_id
host_url
host_name
host_since
host_location
host_about
host_response_time
host_response_rate
host_acceptance_rate
host_is_superhost
host_thumbnail_url
host_picture_url
host_neighbourhood
host_listings_count
host_total_listings_count
host_verifications
host_has_profile_pic
host_identity_verified
street
neighbourhood
neighbourhood_cleansed
neighbourhood_group_cleansed
city
state
zipcode
market
smart_location
country_code
country
latitude
longitude
is_location_exact
property_type
room_type
accommodates
bathrooms
bedrooms
beds
bed_type
amenities
square_feet
price
weekly_price
monthly_price
security_deposit
cleaning_fee
guests_included
extra_people
minimum_nights
maximum_nights
calendar_updated
has_availability
availability_30
availability_60
availability_90
availability_365
cal

In [22]:
listings.dtypes

id                                    int64
listing_url                          object
scrape_id                             int64
last_scraped                         object
name                                 object
summary                              object
space                                object
description                          object
experiences_offered                  object
neighborhood_overview                object
notes                                object
transit                              object
access                               object
interaction                          object
house_rules                          object
thumbnail_url                        object
medium_url                           object
picture_url                          object
xl_picture_url                       object
host_id                               int64
host_url                             object
host_name                            object
host_since                      

In [25]:
listings.count()

id                                  36608
listing_url                         36608
scrape_id                           36608
last_scraped                        36608
name                                36608
summary                             34652
space                               26945
description                         36599
experiences_offered                 36608
neighborhood_overview               22220
notes                               14101
transit                             23670
access                              22327
interaction                         21407
house_rules                         25373
thumbnail_url                       31624
medium_url                          31624
picture_url                         36608
xl_picture_url                      31624
host_id                             36608
host_url                            36608
host_name                           36602
host_since                          36602
host_location                     

In [ ]:
def wrangler(df):
    # Drop non-generic columns: PassengerId, Name, Ticket, and near-empty column Cabin
    df = df.drop(['scrape_id','last_scraped','experiences_offered','has_availability','license','jurisdiction_names', ''], axis=1)

    # Replace missing values for age using median value
    df.loc[(df['Age'].isnull()), 'Age'] = df['Age'].dropna().median()
    
    # Replace missing values for Fare using median value (there are some missing in Kaggle's test set)
    df.loc[(df['Fare'].isnull()), 'Fare'] = df['Fare'].dropna().median()
    
    # Replace missing values for embarked using mode value
    df.loc[(df['Embarked'].isnull()), 'Embarked'] = df['Embarked'].dropna().mode()
    
    return df

In [48]:
listings.isnull().sum()

0

In [51]:
listings.describe

<bound method NDFrame.describe of id                                      0
listing_url                             0
scrape_id                               0
last_scraped                            0
name                                    0
summary                              1956
space                                9663
description                             9
experiences_offered                     0
neighborhood_overview               14388
notes                               22507
transit                             12938
access                              14281
interaction                         15201
house_rules                         11235
thumbnail_url                        4984
medium_url                           4984
picture_url                             0
xl_picture_url                       4984
host_id                                 0
host_url                                0
host_name                               6
host_since                              6
